In [ ]:
##Below Python script that imports several libraries and defines functions related to the YOLOv4 object detection model. 
##Here's a breakdown of each line

import colorsys
#below imports the colorsys module, which provides conversion functions for converting between various color representations, 
#such as RGB and HSV.
import numpy as np
from keras import backend as K
##below line the backend module from the Keras library, which provides an abstraction layer for working with different deep learning frameworks such as TensorFlow, 
##Theano, and CNTK. It is imported with the alias K.
from keras.models import load_model
##This line imports the load_model function from the Keras library, which allows us to load a pre-trained deep learning model 
##saved in the HDF5 file format.
from yolo4.model import yolo_eval, Mish
from yolo4.utils import letterbox_image
import os
from keras.utils import multi_gpu_model

In [ ]:


"""
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
"""

'''

model_path: The path to the pre-trained YOLOv4 model.
anchors_path: The path to the file containing the anchor boxes used by the YOLOv4 model.
classes_path: The path to the file containing the class names used by the YOLOv4 model.
gpu_num: The number of GPUs to use for inference. Default value is 1.
score: The confidence score threshold for object detection. Default value is 0.5.
iou: The intersection over union threshold for object detection. Default value is 0.5.
class_names: A list of the class names used by the YOLOv4 model.
anchors: A numpy array of the anchor boxes used by the YOLOv4 model.
sess: The TensorFlow session used for inference.
model_image_size: The input image size expected by the YOLOv4 model. Default value is (416, 416).
is_fixed_size: A boolean value indicating whether the input image size is fixed or not.
boxes: A TensorFlow placeholder for the bounding box coordinates of the detected objects.
scores: A TensorFlow placeholder for the confidence scores of the detected objects.
classes: A TensorFlow placeholder for the class indices of the detected objects.


'''

class YOLO(object):
    def __init__(self):
        self.model_path = 'model_data/yolo4.h5'
        self.anchors_path = 'model_data/yolo_anchors.txt'
        self.classes_path = 'model_data/coco_classes.txt'
        self.gpu_num = 1
        self.score = 0.5
        self.iou = 0.5
        self.class_names = self._get_class()
        self.anchors = self._get_anchors()
        self.sess = K.get_session()
        self.model_image_size = (416, 416)  # fixed size or (None, None)
        self.is_fixed_size = self.model_image_size != (None, None)
        self.boxes, self.scores, self.classes = self.generate()

    def _get_class(self):
        classes_path = os.path.expanduser(self.classes_path)
        with open(classes_path) as f:
            class_names = f.readlines()
        class_names = [c.strip() for c in class_names]
        return class_names

    def _get_anchors(self):
        anchors_path = os.path.expanduser(self.anchors_path)
        with open(anchors_path) as f:
            anchors = f.readline()
            anchors = [float(x) for x in anchors.split(',')]
            anchors = np.array(anchors).reshape(-1, 2)
        return anchors

    def generate(self):
        model_path = os.path.expanduser(self.model_path)
        assert model_path.endswith('.h5'), 'Keras model or weights must be a .h5 file.'

        self.yolo_model = load_model(model_path, custom_objects={'Mish': Mish}, compile=False)

        print('{} model, anchors, and classes loaded.'.format(model_path))

        # Generate colors for drawing bounding boxes.
        hsv_tuples = [(x / len(self.class_names), 1., 1.)
                      for x in range(len(self.class_names))]
        self.colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
        self.colors = list(
            map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
                self.colors))
        np.random.seed(10101)  # Fixed seed for consistent colors across runs.
        np.random.shuffle(self.colors)  # Shuffle colors to decorrelate adjacent classes.
        np.random.seed(None)  # Reset seed to default.

        # Generate output tensor targets for filtered bounding boxes.
        self.input_image_shape = K.placeholder(shape=(2, ))
        if self.gpu_num>=2:
            self.yolo_model = multi_gpu_model(self.yolo_model, gpus=self.gpu_num)
        boxes, scores, classes = yolo_eval(self.yolo_model.output, self.anchors,
                len(self.class_names), self.input_image_shape,
                score_threshold=self.score, iou_threshold=self.iou)
        return boxes, scores, classes

'''
_get_class(): Reads the class names from the classes file and returns them as a list.
_get_anchors(): Reads the anchor boxes from the anchor file and returns them as a numpy array.
generate(): Loads the pre-trained YOLOv4 model, generates colors for drawing bounding boxes, and creates the TensorFlow placeholders for the bounding box coordinates, confidence scores, and class indices of the detected objects. Returns the placeholders as a tuple.
detect_image(image): Takes an image as input, resizes it to the input size expected by the YOLOv4 model, runs inference using the YOLOv4 model, and returns the bounding box coordinates, confidence scores, and class names of the detected objects as lists.
close_session(): Closes the TensorFlow session used for inference.


'''
    
    def detect_image(self, image):

        if self.is_fixed_size:
            assert self.model_image_size[0]%32 == 0, 'Multiples of 32 required'
            assert self.model_image_size[1]%32 == 0, 'Multiples of 32 required'
            boxed_image = letterbox_image(image, tuple(reversed(self.model_image_size)))
        else:
            new_image_size = (image.width - (image.width % 32),
                              image.height - (image.height % 32))
            boxed_image = letterbox_image(image, new_image_size)
        image_data = np.array(boxed_image, dtype='float32')

        image_data /= 255.
        image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

        out_boxes, out_scores, out_classes = self.sess.run(
            [self.boxes, self.scores, self.classes],
            feed_dict={
                self.yolo_model.input: image_data,
                self.input_image_shape: [image.size[1], image.size[0]],
                K.learning_phase(): 0
            })
        return_boxes = []
        return_scores = []
        return_class_names = []
        for i, c in reversed(list(enumerate(out_classes))):
            predicted_class = self.class_names[c]
            ''''
            if predicted_class != 'person':
                continue
            '''
            box = out_boxes[i]
            score = out_scores[i]
            x = int(box[1])
            y = int(box[0])
            w = int(box[3] - box[1])
            h = int(box[2] - box[0])
            if x < 0 :
                w = w + x
                x = 0
            if y < 0 :
                h = h + y
                y = 0
            return_boxes.append([x, y, w, h])
            return_scores.append(score)
            return_class_names.append(predicted_class)

        return return_boxes, return_scores, return_class_names

    def close_session(self):
        self.sess.close()